# ETL using Python & MS-SQL-Server on Kaggle Orders dataset. <br> Exploring Sales queries using SQL.<br> 

In [ ]:
pip install opendatasets

In [ ]:
pip install sqlalchemy

In [11]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


### Downloading the dataset from Kaggle

In [4]:
import opendatasets as od

dataset_url = 'https://www.kaggle.com/datasets/manoj07d/Furniture-Orders'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: manoj07d
Your Kaggle Key: ········
Dataset URL: https://www.kaggle.com/datasets/manoj07d/Furniture-Orders


100%|████████████████████████████████████████████████████████████████████████████████| 200k/200k [00:00<00:00, 217kB/s]

### Extracting zip file

In [10]:
import zipfile
zipref = zipfile.ZipFile('Furniture-Orders.zip')
zipref.extractall()
zipref.close()

In [2]:
import pandas as pd

In [2]:
df = pd.read_csv('Furniture-Orders\orders.csv')
df.head(20)

Order Id  Order Date       Ship Mode      Segment        Country  \
0          1  2023-03-01    Second Class     Consumer  United States   
1          2  2023-08-15    Second Class     Consumer  United States   
2          3  2023-01-10    Second Class    Corporate  United States   
3          4  2022-06-18  Standard Class     Consumer  United States   
4          5  2022-07-13  Standard Class     Consumer  United States   
5          6  2022-03-13   Not Available     Consumer  United States   
6          7  2022-12-28  Standard Class     Consumer  United States   
7          8  2022-01-25  Standard Class     Consumer  United States   
8          9  2023-03-23   Not Available     Consumer  United States   
9         10  2023-05-16  Standard Class     Consumer  United States   
10        11  2023-03-31   Not Available     Consumer  United States   
11        12  2023-12-25   Not Available     Consumer  United States   
12        13  2022-02-11  Standard Class     Consumer  United States   
13        14  2023-07-18  Standard Class     Consumer  United States   
14        15  2023-11-09         unknown  Home Office  United States   
15        16  2022-06-18  Standard Class  Home Office  United States   
16        17  2022-02-04  Standard Class     Consumer  United States   
17        18  2023-08-04    Second Class     Consumer  United States   
18        19  2022-01-23    Second Class     Consumer  United States   
19        20  2022-01-11    Second Class     Consumer  United States   

               City           State  Postal Code   Region         Category  \
0         Henderson        Kentucky        42420    South        Furniture   
1         Henderson        Kentucky        42420    South        Furniture   
2       Los Angeles      California        90036     West  Office Supplies   
3   Fort Lauderdale         Florida        33311    South        Furniture   
4   Fort Lauderdale         Florida        33311    South  Office Supplies   
5       Los Angeles      California        90032     West        Furniture   
6       Los Angeles      California        90032     West  Office Supplies   
7       Los Angeles      California        90032     West       Technology   
8       Los Angeles      California        90032     West  Office Supplies   
9       Los Angeles      California        90032     West  Office Supplies   
10      Los Angeles      California        90032     West        Furniture   
11      Los Angeles      California        90032     West       Technology   
12          Concord  North Carolina        28027    South  Office Supplies   
13          Seattle      Washington        98103     West  Office Supplies   
14       Fort Worth           Texas        76106  Central  Office Supplies   
15       Fort Worth           Texas        76106  Central  Office Supplies   
16          Madison       Wisconsin        53711  Central  Office Supplies   
17      West Jordan            Utah        84084     West  Office Supplies   
18    San Francisco      California        94109     West  Office Supplies   
19    San Francisco      California        94109     West       Technology   

   Sub Category       Product Id  cost price  List Price  Quantity  \
0     Bookcases  FUR-BO-10001798         240         260         2   
1        Chairs  FUR-CH-10000454         600         730         3   
2        Labels  OFF-LA-10000240          10          10         2   
3        Tables  FUR-TA-10000577         780         960         5   
4       Storage  OFF-ST-10000760          20          20         2   
5   Furnishings  FUR-FU-10001487          50          50         7   
6           Art  OFF-AR-10002833          10          10         4   
7        Phones  TEC-PH-10002275         860         910         6   
8       Binders  OFF-BI-10003910          20          20         3   
9    Appliances  OFF-AP-10002892          90         110         5   
10       Tables  FUR-TA-10001539        1470        1710         9   
11       Pho

### Transform the data

In [31]:
# Replacing 'Not Available','unknown' values with NaN
df = pd.read_csv('Furniture-Orders\orders.csv',na_values=['Not Available', 'unknown'])

In [34]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [23]:
#Correcting datatype of Order date column
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%Y-%m-%d') # %Y in caps
df.dtypes

Order Id                     int64
Order Date          datetime64[ns]
Ship Mode                   object
Segment                     object
Country                     object
City                        object
State                       object
Postal Code                  int64
Region                      object
Category                    object
Sub Category                object
Product Id                  object
cost price                   int64
List Price                   int64
Quantity                     int64
Discount Percent             int64
dtype: object

In [3]:
#Correcting Column names
df.columns = df.columns.str.lower() #Changing all columns to str and to lower case
df.columns = df.columns.str.replace(" ","_")  #Changing all spaces to underscore
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

### Adding some necessary columns

In [4]:
# Deriving Discount col
df['discount'] = df['list_price']*df['discount_percent']*0.01

In [5]:
# Deriving Sales Price col
df['sales_price'] = df['list_price']-df['discount']

In [6]:
# Deriving Profit col
df['profit'] = df['sales_price']-df['cost_price']

### Deleting some unnecessary columns

In [7]:
df.drop(columns=['list_price','cost_price','discount_percent'], inplace=True)

In [8]:
df.head()

order_id  order_date       ship_mode    segment        country  \
0         1  2023-03-01    Second Class   Consumer  United States   
1         2  2023-08-15    Second Class   Consumer  United States   
2         3  2023-01-10    Second Class  Corporate  United States   
3         4  2022-06-18  Standard Class   Consumer  United States   
4         5  2022-07-13  Standard Class   Consumer  United States   

              city       state  postal_code region         category  \
0        Henderson    Kentucky        42420  South        Furniture   
1        Henderson    Kentucky        42420  South        Furniture   
2      Los Angeles  California        90036   West  Office Supplies   
3  Fort Lauderdale     Florida        33311  South        Furniture   
4  Fort Lauderdale     Florida        33311  South  Office Supplies   

  sub_category       product_id  quantity  discount  sales_price  profit  
0    Bookcases  FUR-BO-10001798         2       5.2        254.8    14.8  
1       Chairs  FUR-CH-10000454         3      21.9        708.1   108.1  
2       Labels  OFF-LA-10000240         2       0.5          9.5    -0.5  
3       Tables  FUR-TA-10000577         5      19.2        940.8   160.8  
4      Storage  OFF-ST-10000760         2       1.0         19.0    -1.0

In [11]:
# Writing to a new file
df.to_csv('Furniture-Orders\df_orders.csv',index=False)

### Loading the data to MSSqlServer

In [16]:
import pyodbc

conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=MONOZ;"
    "Database=MyDB;"
    "Trusted_Connection=yes;"
)
conn.connect()
cursor = conn.cursor()
cursor.execute("SELECT top 5 * FROM df_orders")
rows = cursor.fetchall()

OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC SQL Server Driver]Login timeout expired (0) (SQLDriverConnect)')

In [ ]:
#df.to_sql('df_orders', con=conn, index=False, if_exists='append')

## SQL Queries and Data analysis

#### Top 10 highest reveue generating products

In [17]:
query1 = """
select top 10 sub_category, round(sum(sales_price),2) as sales
from df_orders
group by sub_category
order by sales desc
"""
#result = pd.read_sql()
result = pd.read_sql_query(query1, conn)
result

NameError: name 'conn' is not defined

#### Top 10 highest selling products in each region

In [ ]:
query2 = """
with cte as
	(
	select region, sub_category, round(sum(sales_price),2) as sales
	from df_orders
	group by region, sub_category
	)
		select * from
		(
			select *, row_number() over(partition by region order by sales desc) as rn
			from cte
		) A
		where rn<=10
"""
result = pd.read_sql()
result = pd.read_sql_query(query2, conn)
result

#### Month over month growth comparison for 2022 and 2023 sales

In [ ]:
query3 = """
with cte as
	(
		select month(order_date) as sales_months, year(order_date) as sales_years, round(sum(sales_price),2) as sales
		from df_orders
		group by order_date
	)
		select sales_months,
		sum(case when sales_years = 2022 then sales else 0 end) as sales_2022,
		sum(case when sales_years = 2023 then sales else 0 end) as sales_2023
		from cte
		group by sales_months
		order by sales_months
"""
result = pd.read_sql()
result = pd.read_sql_query(query3, conn)
result

#### Highest monthly sales for each category

In [ ]:
query4 = """
with cte as
	(
		select month(order_date) as sales_month, category, round(sum(sales_price),2) as sales
		from df_orders
		group by category, month(order_date)
	)
	select * from
		(
			select *, row_number() over(partition by category order by sales desc) as rn
			from cte
		) A
	where rn=1
"""
result = pd.read_sql()
result = pd.read_sql_query(query4, conn)
result

#### Product with highest growth by profit in 2023 compare to 2022

In [ ]:
query5 = """
with cte as
	(
		select sub_category, year(order_date) as sales_years, round(sum(profit),2) as profit
		from df_orders
		group by sub_category, year(order_date)
	)
	select top 1 *, (profit_2023-profit_2022) as growth
	from(
		select sub_category,
		sum(case when sales_years=2022 then profit else 0 end) as profit_2022,
		sum(case when sales_years=2023 then profit else 0 end) as profit_2023
		from cte
		group by sub_category
		) A
	order by growth desc
"""
result = pd.read_sql()
result = pd.read_sql_query(query5, conn)
result